-----

# **`Model Trainer`**


----

In [1]:
!pwd

/e/Practice python/Chest Cancer Classification Using MLflow and DVC/research


In [2]:
# change directory
import os
os.chdir("../")

In [3]:
!pwd

/e/Practice python/Chest Cancer Classification Using MLflow and DVC


### **Setting Up Training Configrarion**

In [4]:
from dataclasses import dataclass
from pathlib import Path

# Configuration class for training settings
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path  # Directory for storing training artifacts
    trained_model_path: Path  # Path where the trained model will be saved
    updated_base_model_path: Path  # Path to the updated base model
    training_data: Path  # Path to the training dataset
    params_epochs: int  # Number of epochs for training
    params_batch_size: int  # Batch size for training
    params_is_augmentation: bool  # Flag to indicate if data augmentation is used
    params_image_size: list  # Image dimensions for input to the model

In [5]:
import tensorflow as tf
from src.chest_cancer_classifier import *
from src.chest_cancer_classifier.constants import *
from src.chest_cancer_classifier.utils.common_functions import read_yaml, create_directories
from src.chest_cancer_classifier.entity.config_entity import PrepareBaseModelConfig

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,  # Path to the configuration file
        params_filepath=PARAMS_FILE_PATH    # Path to the parameters file
    ):
        # Read the configuration and parameters from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create necessary directories specified in the configuration
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        # Retrieve training configuration details from the loaded config
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        # Define the path to the training data
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        
        # Create the root directory for training if it does not exist
        create_directories([
            Path(training.root_dir)
        ])

        # Create a TrainingConfig object with the relevant parameters
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),  # Root directory for training
            trained_model_path=Path(training.trained_model_path),  # Path to save the trained model
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),  # Path for the updated base model
            training_data=Path(training_data),  # Path to the training data
            params_epochs=params.EPOCHS,  # Number of epochs for training
            params_batch_size=params.BATCH_SIZE,  # Batch size for training
            params_is_augmentation=params.AUGMENTATION,  # Whether to use data augmentation
            params_image_size=params.IMAGE_SIZE  # Size of the images for training
        )

        # Return the constructed training configuration
        return training_config

In [ ]:
import os
import time
import tensorflow as tf
import urllib.request as request
from zipfile import ZipFile

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Load the base model from the specified path
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Compile the model with a new optimizer and appropriate loss
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='sparse_categorical_crossentropy',  # Set based on integer label expectation
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation data generator with integer labels (for sparse categorical crossentropy)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode="sparse",  # Set to "sparse" for integer labels
            **dataflow_kwargs
        )

        # Training data generator with or without augmentation
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            class_mode="sparse",  # Set to "sparse" for integer labels
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Start training
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        # Save the trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [9]:
try:
    # Initialize the ConfigurationManager to load configuration settings
    config = ConfigurationManager()
    
    # Retrieve training configuration from the ConfigurationManager
    training_config = config.get_training_config()
    
    # Create a Training object using the retrieved training configuration
    training = Training(config=training_config)
    
    # Load the base model specified in the training configuration
    training.get_base_model()
    
    # Set up the training and validation data generators
    training.train_valid_generator()
    
    # Start the training process
    training.train()

except Exception as e:
    # Raise any exceptions that occur during the process for further handling
    raise e

[2024-11-11 23:11:49,238: INFO: common_functions: YAML file 'config\config.yaml' loaded successfully]
[2024-11-11 23:11:49,249: INFO: common_functions: YAML file 'params.yaml' loaded successfully]
[2024-11-11 23:11:49,263: INFO: common_functions: Directory created at: artifacts]
[2024-11-11 23:11:49,271: INFO: common_functions: Directory created at: artifacts\training]
[2024-11-11 23:11:50,068: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


e:\Practice python\Chest Cancer Classification Using MLflow and DVC\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6309 - loss: 0.8899

e:\Practice python\Chest Cancer Classification Using MLflow and DVC\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 108s 6s/step - accuracy: 0.6315 - loss: 0.8877 - val_accuracy: 1.0000 - val_loss: 0.0471
[2024-11-11 23:13:40,070: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


----